In [1]:
# Initiate linear demand curve parameters for simulation analysis
# One linear demand curve is initiated for each rate class, arrival day of week and 
# los combination

In [2]:
import itertools
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from sklearn.linear_model import LinearRegression

In [3]:
# Consider two rate classes for simplicity
n_class = 2
# Consider Length-of-Stay (los) of upto three days
los = 3
capacity = 50
# Consider two demand scenario, by changing capacity
demand_intensity = 0.9
# Slope for linear demand curve
slope = np.array([-0.1, -0.15])
# Nightly rates for Monday stay night through Sunday stay night
# Higher weekday rates imply business hotel settings and higher weekend rates imply resorts
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [4]:
# Take the slope of -0.1 for rate class 1, then the mean true demand point for any arrival date/lengthof-
# stay combination is passed through by the line defined by this slope; 30 points on the price 
# axis for each arrival date/length-of-stay combination over the first 21 simulation days 
# are randomly generated from the interval (init_rate-50, init_rate+50)
# Sunday-one night stay as an example
drawsize = 30
# Random draw for one night stay for each day of week
randomRates_one = [np.random.randint(rates_init[i, j]-50, rates_init[i, j]+50+1, drawsize) 
                     for i, j in itertools.product(range(n_class), range(7))]
randomRates_one = np.array(randomRates_one).reshape(n_class, 7, drawsize)
# Check the averages are close to rates_init
np.round(np.average(randomRates_one, axis = 2), decimals=1)

array([[145. , 132.3, 136.6, 142.7, 132.3, 110.9, 112.4],
       [118.3, 122.2, 114.4, 108.5, 120.4,  84.2, 102.3]])

In [5]:
randomRates_one.shape

(2, 7, 30)

In [6]:
# Construct rates for more than one night stay, up to 3-night stay
# Sunday night arrival, two night stay rates equal Sunday one night + Monday one night stay
randomRates_two = [randomRates_one[i, j%7] 
                     + randomRates_one[i, (j+1)%7] 
                     for i, j in itertools.product(range(n_class), range(7))]

randomRates_three = [randomRates_one[i, j%7] 
                       + randomRates_one[i, (j+1)%7] 
                       + randomRates_one[i, (j+2)%7] 
                     for i, j in itertools.product(range(n_class), range(7))]

In [7]:
# For each arrival date, los can be 1-night, 2-night, or 3-night.
# Accordingly, the rates will be corresponding to different los.
# e.g., for Mon arrival, 2-night stay, the rate equals Mon single night rate + Tu single night rate
randomRates_one = np.array(randomRates_one).reshape(n_class, 7, drawsize)
randomRates_two = np.array(randomRates_two).reshape(n_class, 7, drawsize)
randomRates_three = np.array(randomRates_three).reshape(n_class, 7, drawsize)
randomRates = [(randomRates_one[i, j], randomRates_two[i, j], randomRates_three[i, j]) 
               for i, j in itertools.product(range(n_class), range(7))]

In [8]:
# axis 0 = n_class, axis 1 = day of week, axis 2 = los, and axis 3 = random draws
randomRates = np.array(randomRates).reshape(n_class, 7, los, drawsize)
randomRates.shape

(2, 7, 3, 30)

In [9]:
# Randomly generated rates for class 1, Sunday arrivals
randomRates[0, 0]

array([[176, 148, 140, 166, 174, 100, 168, 101, 113, 137, 105, 152, 169,
        131, 180, 176, 166, 175, 117, 120, 185, 179, 125, 135, 119, 166,
         89, 105, 155, 177],
       [307, 248, 291, 278, 269, 197, 267, 253, 257, 299, 262, 296, 335,
        300, 277, 280, 253, 285, 260, 277, 303, 286, 275, 258, 301, 338,
        241, 280, 279, 266],
       [478, 340, 448, 419, 429, 292, 437, 436, 394, 428, 355, 470, 501,
        457, 433, 372, 342, 421, 430, 415, 440, 386, 408, 424, 416, 489,
        363, 417, 405, 371]])

In [10]:
# Check accuracy, result should be similar to init_rate
np.mean(randomRates[1], axis = 2)

array([[118.33333333, 240.53333333, 354.96666667],
       [122.2       , 236.63333333, 345.16666667],
       [114.43333333, 222.96666667, 343.33333333],
       [108.53333333, 228.9       , 313.1       ],
       [120.36666667, 204.56666667, 306.86666667],
       [ 84.2       , 186.5       , 304.83333333],
       [102.3       , 220.63333333, 342.83333333]])

In [11]:
# Construct rates for more than one night stay, up to 3-night stay
rates_init

array([[135, 135, 135, 135, 135, 108, 108],
       [115, 115, 115, 115, 115,  92,  92]])

In [12]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [13]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [14]:
# Calculate y-intercepts, assuming los distribution for a stay night is 1/3, 1/3, and 1/3
# There exist one 1-night stay, two 2-night stays and three 3-night stays that cover 
# stay night in question.
# Each rate class contribute half of the demand for each stay night
half_demand = 0.5 * capacity * demand_intensity
# Intercepts for 1-night stay
A1 = np.array([[1, 0, 0, 0, 0, 0, 0],
               [0, 1, 0, 0, 0, 0, 0],
               [0, 0, 1, 0, 0, 0, 0],
               [0, 0, 0, 1, 0, 0, 0],
               [0, 0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 0, 1]])
b1 = [(half_demand * 1/3 - slope[i] * rates_arrival_los[i, j, 0]) 
               for i, j in itertools.product(range(n_class), range(7))]
b1 = np.array(b1).reshape(n_class, 7)
x1 = [np.linalg.solve(A1, b1[i]) for i in range(n_class)]

# Intercepts for 2-night stay
A2 = np.array([[1, 0, 0, 0, 0, 0, 1],
               [1, 1, 0, 0, 0, 0, 0],
               [0, 1, 1, 0, 0, 0, 0],
               [0, 0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 1, 0, 0],
               [0, 0, 0, 0, 1, 1, 0],
               [0, 0, 0, 0, 0, 1, 1]])
b2 = [(half_demand * 1/3 - slope[i] * (rates_arrival_los[i, j, 1]
                                               +rates_arrival_los[i, (j-1+7)%7, 1])) 
               for i, j in itertools.product(range(n_class), range(7))]
b2 = np.array(b2).reshape(n_class, 7)
x2 = [np.linalg.solve(A2, b2[i]) for i in range(n_class)]

# Intercepts for 3-night stay
A3 = np.array([[1, 0, 0, 0, 0, 1, 1],
               [1, 1, 0, 0, 0, 0, 1],
               [1, 1, 1, 0, 0, 0, 0],
               [0, 1, 1, 1, 0, 0, 0],
               [0, 0, 1, 1, 1, 0, 0],
               [0, 0, 0, 1, 1, 1, 0],
               [0, 0, 0, 0, 1, 1, 1]])
b3 = [(half_demand * 1/3 - slope[i] * (rates_arrival_los[i, j, 2]
                                               +rates_arrival_los[i, (j-1+7)%7, 2]
                                               +rates_arrival_los[i, (j-2+7)%7, 2]))
             for i, j in itertools.product(range(n_class), range(7))]
b3 = np.array(b3).reshape(n_class, 7)
x3 = [np.linalg.solve(A3, b3[i]) for i in range(n_class)]

In [15]:
# Create intercepts matrix for all arrival day-los combinations
# Total 2 * 7 * 3 = 42 intercepts
intercepts = np.concatenate((np.array(x1), np.array(x2), np.array(x3)), axis=1)
intercepts = np.reshape(intercepts, (n_class, los, 7), order='C')
intercepts = np.array([np.transpose(intercepts[i]) for i in range(n_class)])

In [16]:
intercepts

array([[[21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 40.3 ],
        [21.  , 28.05, 37.6 ],
        [18.3 , 25.35, 37.6 ],
        [18.3 , 28.05, 40.3 ]],

       [[24.75, 38.25, 54.25],
        [24.75, 38.25, 54.25],
        [24.75, 38.25, 54.25],
        [24.75, 38.25, 50.8 ],
        [24.75, 34.8 , 47.35],
        [21.3 , 31.35, 47.35],
        [21.3 , 34.8 , 50.8 ]]])

In [17]:
# Mean of Poisson demand that is generated by the slope and y-intercept
# One mean demand for each of the 30 randomly drawn price
meanDemand = [intercepts[i,j, k] + slope[i] * randomRates[i, j, k, l] 
                    for i, j, k, l in itertools.product(range(n_class), range(7), range(los), range(drawsize))]
meanDemand = np.array(meanDemand).reshape(n_class, 7, los, drawsize)
zeros = np.zeros(n_class * 7 * los * drawsize).reshape(n_class, 7, los, drawsize)
# Can't have megative demand, so truncate mean demand at zero
meanDemand = np.maximum(meanDemand, zeros)

In [18]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
np.average(meanDemand[0, 0], axis=1)

array([6.50333333, 3.21666667, 2.57666667])

In [19]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
meanDemand[0, 0, :, 0]

array([3.4 , 0.05, 0.  ])

In [20]:
# Check total mean demand for each control day of week for each random drawn price
totalDemand = [np.sum(meanDemand[i, j,:,k]) 
                     + np.sum(meanDemand[i, (j-1+7)%7, 1:, k]) 
                     + np.sum(meanDemand[i, (j-2+7)%7, 2:, k]) 
                     for i, j, k in itertools.product(range(n_class), range(7), range(drawsize))]
# Check total mean demand for each day of week
# This mean demand should be around half of the expected demand
totalDemand = np.array(totalDemand).reshape(n_class, 7, drawsize)
np.mean(totalDemand, axis=2)

array([[20.63833333, 23.36      , 23.63      , 22.96      , 24.705     ,
        23.92      , 22.32      ],
       [23.785     , 22.84666667, 27.44333333, 31.68833333, 30.07333333,
        31.955     , 25.30333333]])

In [21]:
# Generate random demand for each day of week and length of stay combination
randomDemand = [poisson.rvs(mu, size=1) for mu in np.nditer(meanDemand)]
randomDemand = np.array(randomDemand).reshape(n_class, 7, los, drawsize)

In [22]:
# Class 1 Sunday arrivals for 1-night, 2-night, and 3-night stays
randomDemand[0, 0]

array([[ 4,  9,  3,  4,  7, 10,  2,  5,  9,  8, 11,  8,  2,  5,  2,  6,
         5,  4, 12,  9,  6,  2,  4,  9,  9,  6, 10, 12, 11,  0],
       [ 0,  4,  0,  1,  3,  6,  4,  7,  6,  1,  3,  3,  0,  0,  4,  1,
         5,  0,  7,  2,  0,  0,  2,  5,  2,  0,  6,  2,  3,  3],
       [ 0,  6,  0,  0,  1, 11,  0,  0,  5,  0,  8,  0,  0,  0,  0,  3,
        10,  0,  0,  2,  0,  2,  1,  4,  3,  0,  6,  3,  0, 11]])

In [23]:
# Check number of total arrivals span a stay night in question for each day of week, 
# Consider up to 3-night stay
arrivals = [np.sum(randomDemand[i, j,:,k]) 
            + np.sum(randomDemand[i, (j-1+7)%7, 1:, k]) 
            + np.sum(randomDemand[i, (j-2+7)%7, 2:, k]) 
            for i, j, k in itertools.product(range(n_class), range(7), range(drawsize))]
arrivals = np.array(arrivals).reshape(n_class, 7, drawsize)
np.mean(arrivals, axis = 2)

array([[20.33333333, 24.        , 23.33333333, 23.5       , 24.93333333,
        24.43333333, 22.23333333],
       [23.56666667, 22.6       , 26.73333333, 31.66666667, 29.23333333,
        31.86666667, 25.53333333]])

In [24]:
arrivals.shape

(2, 7, 30)

In [25]:
# Rates and demand for Sunday stay night for rate class 1 
randomRates[0, 0], randomDemand[0, 0]

(array([[176, 148, 140, 166, 174, 100, 168, 101, 113, 137, 105, 152, 169,
         131, 180, 176, 166, 175, 117, 120, 185, 179, 125, 135, 119, 166,
          89, 105, 155, 177],
        [307, 248, 291, 278, 269, 197, 267, 253, 257, 299, 262, 296, 335,
         300, 277, 280, 253, 285, 260, 277, 303, 286, 275, 258, 301, 338,
         241, 280, 279, 266],
        [478, 340, 448, 419, 429, 292, 437, 436, 394, 428, 355, 470, 501,
         457, 433, 372, 342, 421, 430, 415, 440, 386, 408, 424, 416, 489,
         363, 417, 405, 371]]),
 array([[ 4,  9,  3,  4,  7, 10,  2,  5,  9,  8, 11,  8,  2,  5,  2,  6,
          5,  4, 12,  9,  6,  2,  4,  9,  9,  6, 10, 12, 11,  0],
        [ 0,  4,  0,  1,  3,  6,  4,  7,  6,  1,  3,  3,  0,  0,  4,  1,
          5,  0,  7,  2,  0,  0,  2,  5,  2,  0,  6,  2,  3,  3],
        [ 0,  6,  0,  0,  1, 11,  0,  0,  5,  0,  8,  0,  0,  0,  0,  3,
         10,  0,  0,  2,  0,  2,  1,  4,  3,  0,  6,  3,  0, 11]]))

In [26]:
# Create linear regression object
slopes_update = []
intercepts_update = []
regr = LinearRegression()
for i, j, k in itertools.product(range(n_class), range(7), range(los)):
    regr.fit(randomRates[i, j, k, :].reshape(drawsize, -1), randomDemand[i, j, k, :])
    intercepts_update.append(regr.intercept_)
    slopes_update.append(regr.coef_)

In [27]:
slopes_update = np.array(slopes_update).reshape(n_class, 7, los)
intercepts_update = np.array(intercepts_update).reshape(n_class, 7, los)

In [28]:
slopes_update, intercepts_update

(array([[[-0.07594953, -0.06561424, -0.06301721],
         [-0.11992563, -0.09024233, -0.07049887],
         [-0.09138282, -0.09077322, -0.07178556],
         [-0.11293003, -0.08637421, -0.08582598],
         [-0.09960282, -0.09464667, -0.05444705],
         [-0.09876834, -0.0881714 , -0.05147391],
         [-0.09426554, -0.06999039, -0.07450737]],
 
        [[-0.14165646, -0.07633223, -0.10975472],
         [-0.1724553 , -0.09177277, -0.09545327],
         [-0.1364252 , -0.13642381, -0.0897306 ],
         [-0.15950198, -0.10638805, -0.11117524],
         [-0.13291119, -0.11354309, -0.083495  ],
         [-0.1310014 , -0.09378962, -0.10450127],
         [-0.14837   , -0.11453852, -0.06483943]]]),
 array([[[17.47681729, 20.85930807, 28.61405603],
         [24.29949418, 28.49949597, 32.08165077],
         [19.31622694, 29.18326798, 32.37548343],
         [22.94468389, 27.28048394, 36.51452507],
         [21.2408003 , 26.9149151 , 22.55955496],
         [18.25340909, 23.75534037, 21.52279